### Clone Git repo

In [1]:
%cd /root
! git clone https://github.com/myles-i/DLH_TransferLearning.git
%cd DLH_TransferLearning

/root
Cloning into 'DLH_TransferLearning'...
remote: Enumerating objects: 599, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 599 (delta 41), reused 44 (delta 21), pack-reused 527
Receiving objects: 100% (599/599), 4.33 MiB | 10.59 MiB/s, done.
Resolving deltas: 100% (350/350), done.
/root/DLH_TransferLearning


### Install dependencies



In [2]:
%%capture
! pip install -r requirements.txt

### Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Determine path to data and job directories on Drive

Example for me:

`/content/drive/MyDrive/DLHProject`

where `DLHProject` is the name of my shortcut for the shared folder.

In [4]:
# setup top level directory paths
PROJECT_DIR = '/content/drive/MyDrive/DLHProject'
DATA_DIR = PROJECT_DIR + '/data'
JOB_DIR = PROJECT_DIR + '/jobs'

In [ ]:
import os
dryrun = True # change this to False once you have verified everything looks good

# finetune/pretraining data paths (can vary depending on model used)
arch = "resnet18_2d"
batch_size = 128 #for ResNet2D, batch size needs to be 64 on V100
EXPERIMENTAL_BASE_DIR = DATA_DIR + "/physionet_finetune_spectrogram/experimental"
PHYSIONET_TRAIN = DATA_DIR + "/physionet_finetune_spectrogram/physionet_train.pkl"
PHYSIONET_TEST = DATA_DIR + "/physionet_finetune_spectrogram/physionet_test.pkl"
PRETRAIN_BASE =JOB_DIR + "/spectrogram/pretraining/min_normalization_16epochs_to_20percent"
FINETUNE_BASE =JOB_DIR + "/spectrogram/finetuning/min_normalization_16epochs_to_20percent"
os.makedirs(FINETUNE_BASE, exist_ok=True)

# define which combinations of runs to do
# make a list of all directories in DATA_DIR + "/physionet_finetune_spectrogram/experimental
DIRs = os.listdir(EXPERIMENTAL_BASE_DIR)
print(DIRs)
# Loop through each weight type and corresponding weight file
for subdir in DIRs:
    # Assert that the required datasets exist
    PHYSIONET_TRAIN = EXPERIMENTAL_BASE_DIR + f"/{subdir}/physionet_train.pkl"
    PHYSIONET_TEST = EXPERIMENTAL_BASE_DIR + f"/{subdir}/physionet_test.pkl"

    assert os.path.exists(PHYSIONET_TRAIN), f"PHYSIONET_TRAIN does not exist: {PHYSIONET_TRAIN}"
    assert os.path.exists(PHYSIONET_TEST), f"PHYSIONET_TEST does not exist: {PHYSIONET_TEST}"

    # define output dir
    FINETUNE_OUTPUT = FINETUNE_BASE + f"/{subdir}"
    os.makedirs(FINETUNE_OUTPUT, exist_ok=True)

    # Define the command to run
    command = (
        f"python finetune_runner.py \\\n"
        f"--arch {arch} \\\n"
        f"--weights-type random\\\n"
        f"--job-base-dir {FINETUNE_OUTPUT} \\\n"
        f"--train {PHYSIONET_TRAIN} \\\n"
        f"--test {PHYSIONET_TEST} \\\n"
        f"--batch-size 128 \\\n"
        f"--epochs 200 \\\n"
        f"--seed '10' \\\n"
        f"--dryrun {dryrun} \\\n"
    )
    # Execute the command and stream the output here
    print(command)
    ! {command}

seq 10 10 100 | xargs -P 1 -I {} python finetune_runner.py \
--arch resnet18_2d \
--weights-type random \
--weights-file  \
--job-base-dir /content/drive/MyDrive/DLHProject/jobs/spectrogram/finetuning/min_normalization_16epochs_to_20percent \
--train /content/drive/MyDrive/DLHProject/data/physionet_finetune_spectrogram/physionet_train.pkl \
--test /content/drive/MyDrive/DLHProject/data/physionet_finetune_spectrogram/physionet_test.pkl \
--batch-size 128 \
--epochs 200 \
--seed '{}' \
--dryrun False \

Configured command:
python -u -m finetuning.trainer --job-dir /content/drive/MyDrive/DLHProject/jobs/spectrogram/finetuning/min_normalization_16epochs_to_20percent/finetune__random_seed10 --train /content/drive/MyDrive/DLHProject/data/physionet_finetune_spectrogram/physionet_train.pkl --test /content/drive/MyDrive/DLHProject/data/physionet_finetune_spectrogram/physionet_test.pkl --val-size 0.0625 --val-metric f1 --arch resnet18_2d --batch-size 128 --epochs 200 --seed 10
2024-04-26 15:26:0